In [ ]:
# HOSPITAL PATIENT DETERIORATION PREDICTION
# Scenario: ICU Patient Health Risk Forecasting
# A hospital ICU continuously monitors critical patient vitals.
# The goal is to predict the patient’s risk score for the next 6 hours using the previous 12 hours of multivariate time-series data, so doctors can intervene before a medical emergency occurs.
# 🔹 Parameters Collected (Every Hour)
# Feature	Description
# heart_rate	Beats per minute
# systolic_bp	Blood pressure (mmHg)
# oxygen	Oxygen saturation (%)
# resp_rate	Breaths per minute
# temperature	Body temperature (°C)

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [6]:
df = pd.read_csv('vital_data.csv')
df.head()

,Hour,HR,BP,O2,Resp,Temp
0,1,82,120,98,16,36.8
1,2,85,118,97,17,36.9
2,3,88,115,96,18,37.0
3,4,92,112,95,19,37.1
4,5,96,108,94,20,37.2


In [9]:
X = df[['Hour', 'HR', 'BP', 'O2', 'Resp']].values
y = df['Temp'].values

window_size = 12

# Simple train/val split
train_size = int(0.8 * len(X))
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

In [10]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# -----------------------------
# 2. Build Simple RNN model
# -----------------------------
model = Sequential([
    SimpleRNN(
        units=16,
        activation='tanh',          # default, keeps hidden state in [-1,1] [web:72][web:78]
        input_shape=(window_size, 1)
    ),
    Dense(1)                        # predict next temperature
])

model.summary()


c:\Users\Karan.krishna\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 16)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 305 (1.19 KB)

 Trainable params: 305 (1.19 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.01,         # step size for gradient descent [web:49]
    clipnorm=1.0                # clip gradient L2 norm to prevent explosion [web:80]
)

model.compile(optimizer=optimizer,
              loss='mse',       # regression
              metrics=['mae'])

# -----------------------------
# 4. Train the model
# -----------------------------
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=15,
    validation_data=(X_val, y_val),
    verbose=0
)

print("Final train loss:", history.history['loss'][-1])
print("Final val loss  :", history.history['val_loss'][-1])

Final train loss: 0.1544397622346878
Final val loss  : 1.8352428674697876


In [24]:
model_adam = Sequential([
    SimpleRNN(16, activation='tanh', input_shape=(window_size, 1)),
    Dense(1)
])

adam_opt = tf.keras.optimizers.Adam(
    learning_rate=0.01,   # adaptive gradient descent [web:51]
    clipnorm=1.0
)

model_adam.compile(optimizer=adam_opt,
                   loss='mse',
                   metrics=['mae'])


history_adam = model_adam.fit(
    X_train, y_train,
    epochs=200,
    batch_size=2,
    validation_data=(X_val, y_val),
    verbose=0
)

print("Final train loss (Adam):", history_adam.history['loss'][-1])
print("Final val loss  (Adam):", history_adam.history['val_loss'][-1])

Final train loss (Adam): 0.18663784861564636
Final val loss  (Adam): 1.6265586614608765


In [25]:

def forecast_next_n_days(model, last_window, n_days=6):
    """
    model: trained keras model that predicts 1 step ahead
    last_window: 1D array of length window_size (your last observed temps)
    n_days: how many future steps to forecast
    """
    window = last_window.astype(np.float32).copy()
    preds = []

    for _ in range(n_days):
        x = window.reshape(1, len(window), 1)        # (1, window_size, 1)
        yhat = model.predict(x, verbose=0)[0, 0]     # scalar prediction
        preds.append(yhat)
        window = np.append(window[1:], yhat)         # slide window forward

    return np.array(preds)


last_window = y[-window_size:]   # last window_size days (e.g., 12)

pred_6_sgd  = forecast_next_n_days(model, last_window, n_days=6)
pred_6_adam = forecast_next_n_days(model_adam, last_window, n_days=6)

print("Last window:", np.round(last_window, 2))
print("Next 6 days (SGD): ", np.round(pred_6_sgd, 2))
print("Next 6 days (Adam):", np.round(pred_6_adam, 2))


Last window: [36.8 36.9 37.  37.1 37.2 37.4 37.6 37.8 38.  38.3 38.6 39. ]
Next 6 days (SGD):  [36.81 36.96 36.92 36.93 36.93 36.93]
Next 6 days (Adam): [37.13 37.21 37.2  37.2  37.2  37.2 ]
